In [42]:
import numpy as np
import xarray as xr
#import pandas as pd
#from tensorflow.keras.layers import Input, Conv2D, Dense, Flatten, Activation, MaxPool2D
#from tensorflow.keras.models import Model, save_model
import matplotlib.pyplot as plt
import FourierOpticsLib as FO
#from sklearn.preprocessing import MinMaxScaler
%matplotlib inline

In [43]:
ds_path = "/scr/sci/mhayman/holodec/holodec-ml-data/"
ds_name = "synthetic_holograms_v02.nc"
ds = xr.open_dataset(ds_path+ds_name)

In [44]:
ds['hologram_number']

<xarray.DataArray 'hologram_number' (hologram_number: 10000)>
array([   0,    1,    2, ..., 9997, 9998, 9999])
Dimensions without coordinates: hologram_number

In [45]:
# initialize the image Fourier Transform
image_ft = xr.DataArray(np.zeros(ds['image'].shape,dtype='float32'),
                    dims=['hologram_number','xsize','ysize'])

# initialize the particle size bins
diam_bin_edges = np.linspace(20,70,25)  # define the particle size histogram bins
diam_bin_centers = 0.5*(diam_bin_edges[:-1]+diam_bin_edges[1:])
diam_histogram = xr.DataArray(np.zeros((ds['image'].shape[0],diam_bin_centers.size)),
                              coords=[ds['hologram_number'],diam_bin_centers],
                              dims=['hologram_number','diameter_bin_centers'])


In [31]:
diam_bin_centers

array([21.04166667, 23.125     , 25.20833333, 27.29166667, 29.375     ,
       31.45833333, 33.54166667, 35.625     , 37.70833333, 39.79166667,
       41.875     , 43.95833333, 46.04166667, 48.125     , 50.20833333,
       52.29166667, 54.375     , 56.45833333, 58.54166667, 60.625     ,
       62.70833333, 64.79166667, 66.875     , 68.95833333])

In [32]:
ds_ft = ds.assign(image_ft=image_ft)
ds_ft = ds_ft.assign(diam_histogram=diam_histogram)
ds_ft = ds_ft.assign_coords(diam_bin_edges=diam_bin_edges)

In [33]:
ds_ft

<xarray.Dataset>
Dimensions:               (diam_bin_edges: 25, diameter_bin_centers: 24, hologram_number: 10000, particle: 30000, xsize: 600, ysize: 400)
Coordinates:
  * hologram_number       (hologram_number) int64 0 1 2 3 ... 9997 9998 9999
  * diameter_bin_centers  (diameter_bin_centers) float64 21.04 23.12 ... 68.96
  * diam_bin_edges        (diam_bin_edges) float64 20.0 22.08 ... 67.92 70.0
Dimensions without coordinates: particle, xsize, ysize
Data variables:
    hid                   (particle) float32 1.0 1.0 1.0 ... 10000.0 10000.0
    d                     (particle) float32 ...
    x                     (particle) float32 ...
    y                     (particle) float32 ...
    z                     (particle) float32 ...
    image                 (hologram_number, xsize, ysize) uint8 ...
    image_ft              (hologram_number, xsize, ysize) float32 0.0 ... 0.0
    diam_histogram        (hologram_number, diameter_bin_centers) float64 0.0 ... 0.0
Attributes:
    DateCre

In [4]:
# store the Fourier Transform and particle size histogram for each hologram
for im in ds_ft['hologram_number'].values:
    particle_index = np.nonzero(ds_ft['hid'].values==im+1)  # find the particles in this hologram
    
    # make a histogram of particle sizes and store it in the data set
    hist0 = np.histogram(ds_ft['d'].values[particle_index],bins=ds_ft['diam_bin_edges'])
    ds_ft['diam_histogram'][im,:],_ = hist0[0]
    
    # FT the image and store the amplitude of it
    image_ft0 = FO.OpticsFFT(ds_ft['image_ft'][:,:,im]-np.mean(ds_ft['image_ft'][:,:,im]))
    ds_ft['image_ft'][:,:,im] = np.abs(image_ft0)

In [41]:
np.nonzero(ds_ft['hid'].values==1)

(array([0, 1, 2]),)

In [6]:
ds_ft

<xarray.Dataset>
Dimensions:   (hologram_number: 10000, particle: 10000, xsize: 600, ysize: 400)
Dimensions without coordinates: hologram_number, particle, xsize, ysize
Data variables:
    hid       (particle) float32 ...
    d         (particle) float32 ...
    x         (particle) float32 ...
    y         (particle) float32 ...
    z         (particle) float32 ...
    image     (ysize, xsize, hologram_number) uint8 ...
    image_ft  (ysize, xsize, hologram_number) float32 0.71851695 ... 0.8084589
Attributes:
    DateCreated:           28-Oct-2019
    Nx:                    600.0
    Ny:                    400.0
    dx:                    2.96e-06
    dy:                    2.96e-06
    lambda:                3.55e-07
    Dpmin:                 2e-05
    Dpmax:                 7e-05
    zMin:                  0.014
    zMax:                  0.158
    NParticles:            1.0
    Nbits:                 8.0
    ccdelevel:             20000.0
    gainFactor:            24.0
    readN

In [7]:
ds_ft.to_netcdf(ds_path+ds_name.replace('synthethic_holograms','synthethic_holograms_ft_ac_amp'))
#ds_ft.to_netcdf(ds_path+"synthethic_holograms_ft_ac_v0.nc")